Creating the demo table for the change data feed example.

In [0]:
%sql
-- DROP  TABLE IF EXISTS cdc_data.users_change_data_feed


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS cdc_data;
create table if not exists cdc_data.users_change_data_feed
(
  userid int,
  name string, 
  city string, 
  modifiedtime timestamp
)
TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
import random
from datetime import datetime, timedelta

def generate_random_timestamp(start_date, end_date):
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
    end_datetime = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

    start_timestamp = int(start_datetime.timestamp())
    end_timestamp = int(end_datetime.timestamp())
    
    random_timestamp = random.randint(start_timestamp, end_timestamp)
    generated_datetime = datetime.fromtimestamp(random_timestamp)
    
    return generated_datetime

# Example usage
start_date = '2022-01-01 00:00:00'
end_date = '2022-12-31 23:59:59'

random_timestamp = generate_random_timestamp(start_date, end_date)
print(random_timestamp)

2022-03-26 20:27:16


In [0]:
%sql
INSERT INTO cdc_data.users_change_data_feed
VALUES
  -- Initial load.
  (124, 'Raul', 'Oaxaca', '2022-08-04 20:44:23'),
  (123, 'Isabel', 'Monterrey', '2022-10-11 07:50:42'),
  -- New users.
  (125, 'Mercedes', 'Tijuana', '2022-01-14 16:36:13'),
  (126, 'Lily', 'Cancun', '2022-03-26 20:27:16')

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
SELECT * FROM table_changes('cdc_data.users_change_data_feed',0)

userid,name,city,modifiedtime,_change_type,_commit_version,_commit_timestamp
124,Raul,Oaxaca,2022-08-04T20:44:23.000+0000,insert,1,2024-04-20T13:52:50.000+0000
123,Isabel,Monterrey,2022-10-11T07:50:42.000+0000,insert,1,2024-04-20T13:52:50.000+0000
125,Mercedes,Tijuana,2022-01-14T16:36:13.000+0000,insert,1,2024-04-20T13:52:50.000+0000
126,Lily,Cancun,2022-03-26T20:27:16.000+0000,insert,1,2024-04-20T13:52:50.000+0000


In [0]:
%sql
INSERT INTO cdc_data.users_change_data_feed
VALUES (130, 'Raul130', 'Oaxaca', current_timestamp());

UPDATE cdc_data.users_change_data_feed
SET name = 'Raul123', modifiedtime=current_timestamp()
WHERE userid = 123;

num_affected_rows
1


In [0]:
%sql
SELECT * FROM table_changes('cdc_data.users_change_data_feed',1)

userid,name,city,modifiedtime,_change_type,_commit_version,_commit_timestamp
123,Isabel,Monterrey,2022-10-11T07:50:42.000+0000,update_preimage,3,2024-04-20T14:57:33.000+0000
123,Raul123,Monterrey,2024-04-20T14:57:16.194+0000,update_postimage,3,2024-04-20T14:57:33.000+0000
124,Raul,Oaxaca,2022-08-04T20:44:23.000+0000,insert,1,2024-04-20T13:52:50.000+0000
123,Isabel,Monterrey,2022-10-11T07:50:42.000+0000,insert,1,2024-04-20T13:52:50.000+0000
125,Mercedes,Tijuana,2022-01-14T16:36:13.000+0000,insert,1,2024-04-20T13:52:50.000+0000
126,Lily,Cancun,2022-03-26T20:27:16.000+0000,insert,1,2024-04-20T13:52:50.000+0000
130,Raul130,Oaxaca,2024-04-20T14:56:37.920+0000,insert,2,2024-04-20T14:57:11.000+0000


In [0]:
%sql

select * from  cdc_data.users_change_data_feed

userid,name,city,modifiedtime
124,Raul,Oaxaca,2022-08-04T20:44:23.000+0000
123,Raul-123,Monterrey,2024-04-22T07:54:14.665+0000
125,Mercedes,Tijuana,2022-01-14T16:36:13.000+0000
126,Lily,Cancun,2022-03-26T20:27:16.000+0000
130,Raul130,Oaxaca,2024-04-20T14:56:37.920+0000
131,Sam131,Owen,2024-04-22T07:54:09.116+0000


In [0]:
%sql

select * from hive_metastore.cdc_data.customer_silver_scd_type_1 order by userid desc

userid,name,city,modifiedtime,_rescued_data
131,Sam131,Owen,2024-04-22T07:54:09.116Z,null
130,Raul130,Oaxaca,2024-04-20T14:56:37.920Z,null
126,Lily,Cancun,2022-03-26T20:27:16.000Z,null
125,Mercedes,Tijuana,2022-01-14T16:36:13.000Z,null
124,Raul,Oaxaca,2022-08-04T20:44:23.000Z,null
123,Raul-123,Monterrey,2024-04-22T07:54:14.665Z,null


In [0]:
%sql

select * from hive_metastore.cdc_data.customer_silver_scd_type_2 order by userid desc

userid,name,city,modifiedtime,_rescued_data,__START_AT,__END_AT
131,Sam131,Owen,2024-04-22T07:54:09.116Z,null,2024-04-22T07:54:09.116Z,null
130,Raul130,Oaxaca,2024-04-20T14:56:37.920Z,null,2024-04-20T14:56:37.920Z,null
126,Lily,Cancun,2022-03-26T20:27:16.000Z,null,2022-03-26T20:27:16.000Z,null
125,Mercedes,Tijuana,2022-01-14T16:36:13.000Z,null,2022-01-14T16:36:13.000Z,null
124,Raul,Oaxaca,2022-08-04T20:44:23.000Z,null,2022-08-04T20:44:23.000Z,null
123,Raul123,Monterrey,2024-04-20T14:57:16.194Z,null,2024-04-20T14:57:16.194Z,2024-04-22T07:54:14.665Z
123,Isabel,Monterrey,2022-10-11T07:50:42.000Z,null,2022-10-11T07:50:42.000Z,2024-04-20T14:57:16.194Z
123,Raul-123,Monterrey,2024-04-22T07:54:14.665Z,null,2024-04-22T07:54:14.665Z,null


In [0]:
%sql
INSERT INTO cdc_data.users_change_data_feed
VALUES (132, 'Sam132', 'Owen', current_timestamp());

UPDATE cdc_data.users_change_data_feed
SET name = 'Raul-123-update1', modifiedtime=current_timestamp()
WHERE userid = 123;

num_affected_rows
1
